# 7.2 PDE-Constrained Shape Optimization

We want to solve the PDE-constrained shape optimization problem
$$
            \underset{\Omega\subset \mathsf{D}}{\mbox{min}} \; J(u) := \int_\Omega |u-u_d|^2 \; dx
$$
 subject to that $(\Omega,u)$ satisfy
 $$
           \int_\Omega \nabla u \cdot \nabla v \; dx = \int_\Omega f v \; dx \; \quad \text{ for all } v \in H_0^1(\Omega),
$$
where $\Omega \subset \mathbb R^2$ for given $u_d, f \in C^1(\mathbb R^2)$.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry

In [2]:
geo = SplineGeometry()
geo.AddRectangle( (0.2, 0.2), (0.8, 0.8), bcs = ('b','r','t','l'))
mesh = Mesh(geo.GenerateMesh(maxh = 0.1))

In [3]:
#given data of our problem (chosen such that \Omega^* = [0,1]^2 is the optimal shape)
f = CoefficientFunction(2*y*(1-y)+2*x*(1-x))
ud = x*(1-x)*y*(1-y)

grad_f = CoefficientFunction( (f.Diff(x), f.Diff(y) ) )
grad_ud = CoefficientFunction( (ud.Diff(x), ud.Diff(y) ) )

### State equation

In [4]:
fes = H1(mesh, order=2, dirichlet=".*")
u, v = fes.TnT()
gfu = GridFunction(fes)
scene = Draw (gfu, mesh, "state")

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx

fstate = LinearForm(fes)
fstate += f*v*dx

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [5]:
def SolveStateEquation():
    rhs = gfu.vec.CreateVector()
    rhs.data = fstate.vec - a.mat * gfu.vec
    update = gfu.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()) * rhs
    gfu.vec.data += update

In [6]:
a.Assemble()
fstate.Assemble()
SolveStateEquation()
scene.Redraw()

### Adjoint equation

We set up the adjoint equation
$$
    \mbox{Find } p \in H_0^1(\Omega): \int_\Omega \nabla w \cdot \nabla p \, \mbox dx = - \partial_u J(u)(w) \quad \text{ for all } w \in H_0^1(\Omega)
$$
where $u$ is the solution to the state equation. For $J(u) = \int_\Omega |u-u_d|^2 \mbox dx$, we get
$$
    \partial_u J(u)(w) = -2 \int_\Omega (u-u_d)w \,\mbox dx.
$$
However, we can also use the Diff(...) command:

In [7]:
def Cost(u): 
    return (u-ud)**2*dx

p, w = fes.TnT()
gfp = GridFunction(fes)
scene = Draw (gfp, mesh, "adjoint")

fadjoint = LinearForm(fes)
fadjoint += -1*Cost(gfu).Diff(gfu,w)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [8]:
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - a.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update

In [9]:
fadjoint.Assemble()
SolveAdjointEquation()
scene.Redraw()

In [20]:
Draw(gfu-gfp,mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

In [21]:
gfud = list(gfu.vec.data)
gfpd = list(gfp.vec.data)
for gfue,gfpe in zip(gfud,gfpd):
    print(gfue-gfpe)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.009620707236726665
0.012254545944535416
0.011851887949983916
0.009343723350379286
0.006043984316083979
0.009121390041480723
0.011455478315293923
0.01139540281753879
0.008325308999918849
0.008704379118447117
0.011492846929100676
0.011109772180678143
0.009530232290497844
0.00657467019502536
0.00963784860576135
0.010608525691709551
0.010842101329021378
0.01977220484032261
0.01725862797425328
0.013350381901781309
0.01925672757810752
0.017813927545330697
0.01570569496129353
0.017040176974747977
0.019860444131930527
0.01511887781021652
0.017226227023884315
0.017767908373753683
0.018014285667571163
0.021393496466619102
0.02174645379202496
0.02277804955012286
0.0
0.0
0.0
0.0
0.008202540198268944
0.0
0.0
0.0
0.0
0.008789865872684237
0.0
-0.021950159057346603
-0.004948335176754645
0.0
-0.016345266061121792
-0.004443487551973481
0.0
-0.00747336405545632
-0.005434533445660002
0.0
-0.003520924294101413

In [35]:
l = len(gfud)
tfunc = GridFunction(fes)
tvec = tfunc.vec.CreateVector()
for c in range(0,l):
    print(gfud[c]-gfpd[l-c-1])
    tvec.data[c] = abs(gfud[c]-gfpd[l-c-1])

0.000408289490470634
0.0005480499541829699
0.0003169113632459708
0.0005256921675022454
0.0003965722700794127
0.00032443424741338206
0.00029586255557341473
0.00018687081376986958
0.0003403184169573071
0.000239543613394453
0.0006683666334452552
0.00035382546838221484
0.0002871056823181453
0.00022815524186652018
0.00030414680308936637
0.0003183159418927136
0.0004893105307598467
0.00037193935516634604
0.0004194392567216958
0.0006448086670365487
0.00033581426351362605
0.00030879125156118024
0.00019868780581886158
0.0002029612887419669
0.01077838016465806
0.01365194804560793
0.013385871547486449
0.01061605518762742
0.00686503954450062
0.010291695270730967
0.01267490180593997
0.013005930036792745
0.00944442084286444
0.009808735509162407
0.012677537048069627
0.012361307330463662
0.010724318779180021
0.00732657913206929
0.010571950171671701
0.011808625813804752
0.012174957186754326
0.021684377255211697
0.019019345424633242
0.015171220378236658
0.021231377304591324
0.019522414223613786
0.0175504

In [36]:
tfunc.vec.data += tvec
Draw(tfunc,mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

Note that (for linear problems) the operator on the left hand side of the adjoint equation is just the transpose of the state operator.

### Shape derivative

The shape derivative of the shape function 
$$
\mathcal{J}(\Omega):= \int_\Omega |u-u_d|^2\;dx
$$
at $\Omega$ in direction $X$ is given by the formula
$$
    \begin{array}{rl}
    d\mathcal J(\Omega; X) =&\int_{\Omega}  \mbox{div}(X) |u-u_d|^2  - 2(u-u_d)\nabla u_d \cdot X \, dx \\
		& + \int_{\Omega} (\mbox{div}(X) I - D X - D X^\top )\nabla u \cdot \nabla p \, dx \\
		&- \int_\Omega (\nabla f\cdot X + f \mbox{div}(X)) p\, dx.
    \end{array}
$$
We now assemble this shape derivaitve in NGSolve as follows:

In [ ]:
VEC = H1(mesh, order=2, dim=2)
PHI, X = VEC.TnT()

dJOmega = LinearForm(VEC)
dJOmega += SymbolicLFI(div(X)*( (gfu-ud)**2 - f*gfp + InnerProduct(grad(gfu),grad(gfp))))
dJOmega += SymbolicLFI(-2*(gfu-ud)*InnerProduct(grad_ud,X) - InnerProduct(grad_f,X)*gfp)
dJOmega += SymbolicLFI(-InnerProduct(X.Deriv()*grad(gfu),grad(gfp))-InnerProduct(grad(gfu),X.Deriv()*grad(gfp)))

### Descent Direction

<ul>
 <li> Next, we want to find a vector field $X$ which yields a decrease of the objective function, i.e. we want to find a vector field $X$ such that $$d \mathcal J(\Omega;X)<0.$$ </li>
<li> This can be achieved by solving an auxiliary boundary value problem of the type 
$$
    \mbox{Find } X \in H: \qquad b(X, \Phi) = d\mathcal J(\Omega; \Phi) \; \text{ for all } \Phi \in H
$$
for a suitable Hilbert space $H$ (e.g. $H=H^1(\Omega)^2$). Here, $b(\cdot, \cdot): H \times H \rightarrow \mathbb R$ is a positive definite bilinear form which we are free to choose. Then, $-X$ is a descent direction since $$ d\mathcal J(\Omega; -X) = -d\mathcal J(\Omega; X) = - b(X, X) < 0.$$ </li>
</ul>

In [ ]:
b = BilinearForm(VEC)
b += InnerProduct(grad(X),grad(PHI))*dx + InnerProduct(X,PHI)*dx

gfX = GridFunction(VEC)

# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

In [ ]:
def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [ ]:
b.Assemble()
dJOmega.Assemble()
SolveDeformationEquation()
Draw(-gfX, mesh, "gfX")

Now let us update the geometry in the direction of $-X$. Here, we need to choose the step size accordingly (a descent is only guaranteed for this step size being sufficiently small)

In [ ]:
print('Cost at initial design', Integrate (Cost(gfu), mesh))
gfset.Set((0,0))
scale = 0.5 / Norm(gfX.vec)
gfset.vec.data -= scale * gfX.vec

Let us now update the design and compute the new value of the cost function:

In [ ]:
scene = Draw(gfset)
mesh.SetDeformation(gfset)
scene.Redraw()

a.Assemble()
fstate.Assemble()
SolveStateEquation()
print('Cost at new design', Integrate (Cost(gfu), mesh))

In [ ]:
#reset the design
gfset.Set((0,0))
mesh.SetDeformation(gfset)

#check if initial value of cost function 0.000486578350214552 is recovered
a.Assemble()
fstate.Assemble()
SolveStateEquation()
print('Cost at new design', Integrate (Cost(gfu), mesh))

Finally, this can be applied in an iterative algorithm with a line search for choosing the step size:

In [ ]:
scene = Draw(gfset)

In [ ]:
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
a.Assemble()
fstate.Assemble()
SolveStateEquation()

LineSearch = True

iter_max = 600
Jold = Integrate(Cost(gfu), mesh)
converged = False

# input("Press enter to start optimization")
for k in range(iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    
    print('cost at iteration', k, ': ', Jold)
    
    a.Assemble()
    fstate.Assemble()
    SolveStateEquation()
    
    fadjoint.Assemble()
    SolveAdjointEquation()
    
    b.Assemble()
    dJOmega.Assemble()
    SolveDeformationEquation()

    scale = 0.01 / Norm(gfX.vec)
    gfsetOld = gfset
    gfset.vec.data -= scale * gfX.vec
    
    Jnew = Integrate(Cost(gfu), mesh)
    
    if LineSearch:
        while Jnew > Jold and scale > 1e-12:
            #input('a')
            scale = scale / 2
            print("scale = ", scale)
            if scale <= 1e-12:
                converged = True
                break

            gfset.vec.data = gfsetOld.vec - scale * gfX.vec
            mesh.SetDeformation(gfset)
            
            a.Assemble()
            fstate.Assemble()
            SolveStateEquation()
            Jnew = Integrate(Cost(gfu), mesh)
    
    if converged==True:
        print("No more descent can be found")
        break
    Jold = Jnew

    Redraw(blocking=True)